In [348]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor,  plot_tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,  f1_score, r2_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from tqdm import tqdm 
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.compose import make_column_transformer, make_column_selector
import numpy as np
from sklearn.impute import SimpleImputer

In [349]:
df = pd.read_csv('train.csv')

In [350]:
df

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,1000001,P00069042,F,0-17,10,A,2,0,3,NaN,NaN,8370
1,1000001,P00248942,F,0-17,10,A,2,0,1,6.0,14.0,15200
2,1000001,P00087842,F,0-17,10,A,2,0,12,NaN,NaN,1422
3,1000001,P00085442,F,0-17,10,A,2,0,12,14.0,NaN,1057
4,1000002,P00285442,M,55+,16,C,4+,0,8,NaN,NaN,7969
...,...,...,...,...,...,...,...,...,...,...,...,...
550063,1006033,P00372445,M,51-55,13,B,1,1,20,NaN,NaN,368
550064,1006035,P00375436,F,26-35,1,C,3,0,20,NaN,NaN,371
550065,1006036,P00375436,F,26-35,15,B,4+,1,20,NaN,NaN,137
550066,1006038,P00375436,F,55+,1,C,2,0,20,NaN,NaN,365


In [351]:
X , y = df.drop(['User_ID', 'Product_ID', 'Purchase'], axis =1) , df['Purchase']
X.dtypes

Gender                         object
Age                            object
Occupation                      int64
City_Category                  object
Stay_In_Current_City_Years     object
Marital_Status                  int64
Product_Category_1              int64
Product_Category_2            float64
Product_Category_3            float64
dtype: object

In [352]:
X['Product_Category_1'] = X['Product_Category_1'].astype('str')
X['Product_Category_2'] = X['Product_Category_2'].astype('str')
X['Product_Category_3'] = X['Product_Category_3'].astype('str')


In [353]:
X

,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3
0,F,0-17,10,A,2,0,3,nan,nan
1,F,0-17,10,A,2,0,1,6.0,14.0
2,F,0-17,10,A,2,0,12,nan,nan
3,F,0-17,10,A,2,0,12,14.0,nan
4,M,55+,16,C,4+,0,8,nan,nan
...,...,...,...,...,...,...,...,...,...
550063,M,51-55,13,B,1,1,20,nan,nan
550064,F,26-35,1,C,3,0,20,nan,nan
550065,F,26-35,15,B,4+,1,20,nan,nan
550066,F,55+,1,C,2,0,20,nan,nan


In [354]:
X_train, X_test, Y_train, Y_test = train_test_split(X,y,test_size=0.3, random_state=24)
X_train.dtypes

Gender                        object
Age                           object
Occupation                     int64
City_Category                 object
Stay_In_Current_City_Years    object
Marital_Status                 int64
Product_Category_1            object
Product_Category_2            object
Product_Category_3            object
dtype: object

In [355]:
X_ohe = OneHotEncoder(sparse_output=False,drop='first', handle_unknown='ignore').set_output(transform='pandas')
ct = make_column_transformer((X_ohe,make_column_selector(dtype_include=object)),
                            ('passthrough',make_column_selector(dtype_exclude=object)),
                             verbose_feature_names_out=False).set_output(transform='pandas')

X_trn = ct.fit_transform(X_train)
X_tst = ct.transform(X_test)

In [356]:
X_trn.shape

(385047, 66)

In [357]:
features = [3,5,6,7]
scores = []

for i in tqdm(range(len(features))):
    rf = RandomForestRegressor(n_estimators=20, max_features=features[i])
    rf.fit(X_trn,Y_train)
    y_pred = rf.predict(X_tst)
    scores.append([features[i],r2_score(Y_test,y_pred)])
print(scores)

df1 = pd.DataFrame(scores, columns = ['features','r2_score'])
df1.sort_values('r2_score', ascending = False).iloc[0]

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:50<00:00, 12.66s/it]

[[3, 0.6237902464996956], [5, 0.6263596549016728], [6, 0.62741252932121], [7, 0.6271276181512205]]


features    6.000000
r2_score    0.627413
Name: 2, dtype: float64

In [358]:
test = pd.read_csv('test_Vges7qu.csv')

In [359]:
test.dtypes

Comb                           object
User_ID                         int64
Product_ID                     object
Gender                         object
Age                            object
Occupation                      int64
City_Category                  object
Stay_In_Current_City_Years     object
Marital_Status                  int64
Product_Category_1              int64
Product_Category_2            float64
Product_Category_3            float64
dtype: object

In [360]:
test['Product_Category_1'] = test['Product_Category_1'].astype('str')
test['Product_Category_2'] = test['Product_Category_2'].astype('str')
test['Product_Category_3'] = test['Product_Category_3'].astype('str')

In [361]:
test.drop(['Comb','User_ID','Product_ID'], axis = 1)

,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3
0,M,46-50,7,B,2,0,1,11.0,nan
1,M,26-35,17,C,0,0,3,5.0,nan
2,F,36-45,1,B,4+,0,5,14.0,nan
3,F,36-45,1,B,4+,0,4,9.0,nan
4,F,26-35,1,C,1,0,4,5.0,12.0
...,...,...,...,...,...,...,...,...,...
233594,F,26-35,15,B,4+,0,8,nan,nan
233595,F,26-35,15,B,4+,0,5,8.0,nan
233596,F,26-35,15,B,4+,0,1,5.0,12.0
233597,F,46-50,1,C,4+,0,10,16.0,nan


In [362]:
si = SimpleImputer(strategy='constant', fill_value='Unknown').set_output(transform='pandas')
test[['Product_Category_2','Product_Category_3']] = si.fit_transform(test[['Product_Category_2','Product_Category_3']])

In [363]:
test

,Comb,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3
0,1000004P00128942,1000004,P00128942,M,46-50,7,B,2,0,1,11.0,nan
1,1000009P00113442,1000009,P00113442,M,26-35,17,C,0,0,3,5.0,nan
2,1000010P00288442,1000010,P00288442,F,36-45,1,B,4+,0,5,14.0,nan
3,1000010P00145342,1000010,P00145342,F,36-45,1,B,4+,0,4,9.0,nan
4,1000011P00053842,1000011,P00053842,F,26-35,1,C,1,0,4,5.0,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...
233594,1006036P00118942,1006036,P00118942,F,26-35,15,B,4+,0,8,nan,nan
233595,1006036P00254642,1006036,P00254642,F,26-35,15,B,4+,0,5,8.0,nan
233596,1006036P00031842,1006036,P00031842,F,26-35,15,B,4+,0,1,5.0,12.0
233597,1006037P00124742,1006037,P00124742,F,46-50,1,C,4+,0,10,16.0,nan


In [393]:
X_ohe = OneHotEncoder(sparse_output=False,drop='first', handle_unknown='ignore').set_output(transform='pandas')
ct = make_column_transformer((X_ohe,make_column_selector(dtype_include=object)),
                            ('passthrough',make_column_selector(dtype_exclude=object)),
                             verbose_feature_names_out=False).set_output(transform='pandas')

X_trn = ct.fit_transform(X)

In [395]:
test1= ct.transform(test)

In [397]:
test1.shape




(233599, 66)

In [416]:
rf = RandomForestRegressor(n_estimators=20,max_features=6)
rf.fit(X_trn,y)
y_pred = rf.predict(test1)

df1 = pd.DataFrame(y_pred,columns = ['Purchase'])
df1['User_ID'] = test['User_ID']
df1['Product_ID'] = test['Product_ID']

In [417]:
df1

,Purchase,User_ID,Product_ID
0,15920.733333,1000004,P00128942
1,10491.991667,1000009,P00113442
2,6271.025981,1000010,P00288442
3,2280.850000,1000010,P00145342
4,2328.282143,1000011,P00053842
...,...,...,...
233594,7900.925283,1006036,P00118942
233595,6508.792672,1006036,P00254642
233596,10929.933333,1006036,P00031842
233597,22992.350000,1006037,P00124742


In [422]:
submit = df1.to_csv('Submit.csv', index=False)